In [1]:
# -*- coding: utf-8 -*- 
import os 
import argparse
import re 
import time 

from modules import folderMoniter, patternRecognizer, metaExtracter
from modules import urlDownload, dropboxInteractor, note_modified

In [2]:
# init 
pattern_recog = patternRecognizer(r'- \[.*\]')  # 检测 - [DOI], 或者- [arxivId]
meta_extracter = metaExtracter()
url_download = urlDownload()

In [11]:
md_file = r"C:\Users\dell\Desktop\Notes&Papers\多模态预训练论文笔记.md"
save_folder = r"C:\Users\dell\Desktop\Notes&Papers\Papers\计算机视觉&自然语言处理论文\多模态"

In [12]:
pdf_link_prefix = "./Papers/计算机视觉&自然语言处理论文/多模态/"#最后面一定要有 /

In [13]:
with open(md_file, 'r', encoding='utf-8') as f:
    content = f.read()

In [14]:
m = pattern_recog.findall(content)
print(m)

if m:
    replace_dict = dict()
    for literature in m:
        literature_id = literature.split('[')[-1].split(']')[0]
    
        try:
            bib_dict = meta_extracter.id2bib(literature_id)
            print(bib_dict)
            
            if "pdf_link" in bib_dict.keys():
                pdf_dict = url_download.fetch(bib_dict["pdf_link"])
                if not pdf_dict:
                    pdf_dict = url_download.fetch(literature_id)
            else:
                pdf_dict = url_download.fetch(literature_id)
                
            print("文件的网址是")
            print(pdf_dict['url'])
            
            # Upload attachment and generate shared link
            if "\n" in bib_dict["title"]:
                bib_dict["title"] = re.sub(r' *\n *', ' ', bib_dict["title"])
                
            #保存文件需要改变命名格式
            if ":" in bib_dict["title"]:
                pdf_name = '['+bib_dict['year'][2:]+'] '+bib_dict["title"].replace(':',"")+'.pdf'
            elif "?" in bib_dict["title"]:
                pdf_name = '['+bib_dict['year'][2:]+'] '+bib_dict["title"].replace('?',"")+'.pdf'
            else:
                pdf_name = '['+bib_dict['year'][2:]+'] '+bib_dict["title"]+'.pdf'
                
            
            #保存pdf文件
            save_name = os.path.join(save_folder,pdf_name)
            with open(save_name, "wb") as code:
                 code.write(pdf_dict['pdf'])
            
            
            #得到相对路径，用于在markdown中创立软链接 
            pdf_shared_link = pdf_link_prefix+pdf_name
                
            if 'cited_count' in bib_dict.keys():
                replaced_literature = "- **{}**. {} et.al. **{}**, **{}**, ([pdf]({}))([link]({})), (Citations **{}**).".format(
                    bib_dict['title'], bib_dict["author"].split(" and ")[0], bib_dict['journal'], 
                    bib_dict['year'], pdf_shared_link, bib_dict['url'], bib_dict["cited_count"]
                    )
            else:
                replaced_literature = "- **{}**. {} et.al. **{}**, **{}**, ([pdf]({}))([arxiv]({})), (Citations ).".format(
                    bib_dict['title'], bib_dict["author"].split(" and ")[0], bib_dict['journal'], 
                    bib_dict['year'], pdf_shared_link, bib_dict['url']
                    )
                
            print(replaced_literature)
            replace_dict[literature] = replaced_literature
        except:
            print("")
        
            # replace_dict[literature] = literature
            
    # Modified note
    note_modified(pattern_recog, md_file, **replace_dict)

['- [2109.04448]']
{'journal': 'arxiv', 'url': 'http://arxiv.org/abs/2109.04448v1', 'title': 'Vision-and-Language or Vision-for-Language? On Cross-Modal Influence in\n  Multimodal Transformers', 'year': '2021.09', 'author': 'Stella Frank and Emanuele Bugliarello and Desmond Elliott', 'ENTRYTYPE': 'article'}
文件的网址是
https://arxiv.org/pdf/2109.04448.pdf
- **Vision-and-Language or Vision-for-Language? On Cross-Modal Influence in Multimodal Transformers**. Stella Frank et.al. **arxiv**, **2021.09**, ([pdf](./Papers/计算机视觉&自然语言处理论文/多模态/[21.09] Vision-and-Language or Vision-for-Language On Cross-Modal Influence in Multimodal Transformers.pdf))([arxiv](http://arxiv.org/abs/2109.04448v1)), (Citations ).
